In [ ]:
import os
import sys

src_dir = os.path.abspath("../src")
if src_dir not in sys.path:
   sys.path.append(src_dir)

In [1]:
# import the needed packages
import matplotlib.pyplot as plt
import numpy as np
import tools
import importlib
importlib.reload(tools)

<module 'tools' from '/home/evgenii/ar_problem/fidelity_test/tools.py'>

In [93]:
# create bath

import solver
import importlib
importlib.reload(solver)

# chain size
ns = 30
# on-site energies
e = [1.0]*ns
# hoppings between sites
h = [0.05]*(ns-1)

In [94]:
# discretize the bath: apply the lightcone method
bth = solver.bath.from_chain(chain = (e, h), rel_tol = 10**(-8), ring_size = 5, t_max = 100, visualize = True)

In [95]:
bth.I_rel

array([ 3.82226204e+01,  3.24946857e+01,  2.09438833e+01,  7.19398453e+00,
        1.06964500e+00,  8.00242914e-02,  3.75964630e-03,  1.25098405e-04,
        3.13558596e-06,  6.15656931e-08,  9.74154822e-10,  1.26882635e-11,
        1.32461077e-13,  3.29432670e-17,  1.74769104e-18,  9.42510642e-19,
        2.71976630e-22,  2.23417943e-22,  3.69259838e-24,  6.83398577e-26,
        1.91917689e-26,  6.13387106e-29, -2.61725826e-36, -1.60675773e-34,
       -3.63998151e-32, -2.40156106e-31, -4.76995717e-31, -9.61643610e-20,
       -7.57367195e-17, -1.86143748e-15])

In [96]:
bth.I_rel / bth.I_rel[0]

array([ 1.00000000e+00,  8.50142804e-01,  5.47944727e-01,  1.88212751e-01,
        2.79846068e-02,  2.09363697e-03,  9.83618147e-05,  3.27288928e-06,
        8.20348245e-08,  1.61071356e-09,  2.54863432e-11,  3.31956925e-13,
        3.46551533e-15,  8.61878822e-19,  4.57239986e-20,  2.46584518e-20,
        7.11559353e-24,  5.84517600e-24,  9.66076723e-26,  1.78794277e-27,
        5.02105003e-28,  1.60477513e-30, -6.84740667e-38, -4.20368282e-36,
       -9.52310823e-34, -6.28308848e-33, -1.24794091e-32, -2.51590184e-21,
       -1.98146330e-18, -4.86998918e-17])

In [35]:
bth.n_rel

9

In [36]:
from evolution_chained2_kicked import evolution_chained2_kicked
from tools import mv

num_modes = 30
max_num_quanta = 4
m = tools.spin_boson_model(num_modes, max_num_quanta)

wq = 1.0
Hs = wq * m.s_p @ m.s_m
g = 0.1
V = g * m.s_m
V_dag = g * m.s_p

def f(ti):
    return(0.1*np.cos((ti + 0.5)*bth.dt))

psi = np.zeros(m.dimension, dtype = complex)
psi_mid = np.zeros(m.dimension, dtype = complex)
psi_mid_next = np.zeros(m.dimension, dtype = complex)
psi_buff = np.zeros(m.dimension, dtype = complex)

psi_ini = np.zeros(m.dimension, dtype = complex)
psi_ini[0] = 1
psi_begin = np.copy(psi_ini)

Ht = None

def begin_step(ti, psi):
    Hint = V_dag @ sum(bth.couplings_rel[:, ti] * m.a_dag)
    Hint = Hint + Hint.conj().transpose()
    global Ht
    Ht = Hs + m.s_x * f(ti) + Hint
    
def apply_H(ti, psi_in, psi_out):
    mv(Ht, psi_in, psi_out)

def eval_o(ti, psi):
    pass

first_in_chain = 1
evolution_chained2_kicked(0, bth.nt, bth.dt, begin_step, apply_H, eval_o, psi_begin, psi, psi_mid, psi_mid_next, first_in_chain)

(array([1.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j]),
 array([ 3.23076670e-01-1.57460371e-02j, -1.73461727e-11+9.76304736e-12j,
         6.91274180e-19+1.72277584e-20j, ...,
        -4.03010105e-02+5.67980374e-02j, -2.56954320e-03+5.72471590e-03j,
         2.13610844e-02-3.68646407e-02j]),
 array([ 3.22949195e-01-1.57542865e-02j, -1.71365978e-11+9.78610044e-12j,
         6.92171073e-19+1.72937019e-20j, ...,
        -4.05824755e-02+5.65180623e-02j, -2.60297334e-03+5.71096499e-03j,
         2.15473155e-02-3.67771089e-02j]),
 array([ 3.22949195e-01-1.57542865e-02j, -1.71365978e-11+9.78610044e-12j,
         6.92171073e-19+1.72937019e-20j, ...,
        -4.05824755e-02+5.65180623e-02j, -2.60297334e-03+5.71096499e-03j,
         2.15473155e-02-3.67771089e-02j]))

In [37]:
Nq = sum([m.a_dag[i] @ m.a[i] for i in range(20)])

In [38]:
psi_full = np.copy(psi) / np.sqrt(np.vdot(psi, psi))

In [39]:
np.vdot(psi_full, Nq @ psi_full)

(2.3991238909730725+0j)

In [89]:
from evolution_chained2_kicked import evolution_chained2_kicked
from tools import mv

num_modes = 30
max_num_quanta = 4
m = tools.spin_boson_model(num_modes, max_num_quanta)

n_rel = 11

wq = 1.0
Hs = wq * m.s_p @ m.s_m
g = 0.1
V = g * m.s_m
V_dag = g * m.s_p

def f(ti):
    return(0.1*np.cos((ti + 0.5)*bth.dt))

psi = np.zeros(m.dimension, dtype = complex)
psi_mid = np.zeros(m.dimension, dtype = complex)
psi_mid_next = np.zeros(m.dimension, dtype = complex)
psi_buff = np.zeros(m.dimension, dtype = complex)

psi_ini = np.zeros(m.dimension, dtype = complex)
psi_ini[0] = 1
psi_begin = np.copy(psi_ini)

Ht = None

def begin_step(ti, psi):
    Hint = V_dag @ sum(bth.couplings_rel[: n_rel, ti] * m.a_dag[: n_rel])
    Hint = Hint + Hint.conj().transpose()
    global Ht
    Ht = Hs + m.s_x * f(ti) + Hint
    
def apply_H(ti, psi_in, psi_out):
    mv(Ht, psi_in, psi_out)

def eval_o(ti, psi):
    pass

first_in_chain = 1
evolution_chained2_kicked(0, bth.nt, bth.dt, begin_step, apply_H, eval_o, psi_begin, psi, psi_mid, psi_mid_next, first_in_chain)

(array([1.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j]),
 array([ 0.32307667-0.01574604j,  0.        +0.j        ,
         0.        +0.j        , ..., -0.04030101+0.05679804j,
        -0.00256954+0.00572472j,  0.02136108-0.03686464j]),
 array([ 0.32294919-0.01575429j,  0.        +0.j        ,
         0.        +0.j        , ..., -0.04058248+0.05651806j,
        -0.00260297+0.00571096j,  0.02154732-0.03677711j]),
 array([ 0.32294919-0.01575429j,  0.        +0.j        ,
         0.        +0.j        , ..., -0.04058248+0.05651806j,
        -0.00260297+0.00571096j,  0.02154732-0.03677711j]))

In [90]:
psi_rel = np.copy(psi) / np.sqrt(np.vdot(psi, psi))

In [91]:
np.vdot(psi_rel, Nq @ psi_rel)

(2.3991238909730788+1.734723475976807e-18j)

In [92]:
f = 1 - np.abs(np.vdot(psi_rel, psi_full))
f

1.3988810110276972e-14